# Check data-cleaning with the base models
## Refactoring the code
-  PrepareColsBasePipe => StandardOHETransformer (enebleing auto detection of numerical and categorical columns)




### Base model submission total 570 plases
- 407 -> 0.14312
- Public Score: 0.14431  Privat Score: 0.15419 
- 408 -> 0.14279

### Base model -Cleaner submission 
- 396 -> 0.14621
- Public Score:  0.14587  Privat Score: 0.15537
- 397 -> 0.14534

### Model Clearner + Price difference + SVC this notebook

Public Score:  0.14311  Privat Score: 0.15176


## 1. Imports 

In [ ]:
## add current directory to system path for relative import of the castom pakage: avp_pckg
import sys
import os

## geting one level above current path 
path_absolute = os.path.dirname(os.getcwd()) 

if path_absolute in sys.path:
    print(f"alrady in sys.path: {path_absolute}")
else:
    sys.path.append(os.path.dirname(os.getcwd()))
    print(f"added to sys.path: {path_absolute}"  )



In [ ]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report  # f1_score,  accuracy_score, recall_score, precision_score

## castom packages 
from avp_pckg.avp_model_selection import plot_scores, print_scores
from avp_pckg.avp_model_selection import cross_validate_transformer

from avp_pckg.small_functions import unanimity
from avp_pckg.TransformerCols import StandardOHETransformer
from avp_pckg.IsBadBuy_imputePrices import Cleaner, IsBadBuy_prepare_cols





In [ ]:
## switch on if modifications in avp_pckg are requared for auto-reload of the pakages
%load_ext autoreload
%autoreload 2

## 2. Load data as train and test sets.


In [ ]:
features_train = pd.read_csv(path_absolute + '\\data\\features_train.csv', parse_dates=['PurchDate'], index_col=0)
features_test = pd.read_csv(path_absolute + '\\data\\features_test.csv', parse_dates=['PurchDate'], index_col=0)

target_train = pd.read_csv(path_absolute + '\\data\\target_train.csv', index_col=0)
target_test = pd.read_csv(path_absolute + '\\data\\target_test.csv', index_col=0)

print(features_train.shape, target_train.shape)
features_train.columns

In [ ]:
## set dt cols to str 
## ['VehicleAge','VehYear', 'BYRNO', 'VNST', 'VNZIP1', 'IsOnlineSale'] 
## delete cols with redundant information
## ['PurchDate', 'VehYear', 'WheelTypeID',  'TopThreeAmericanName', 'VNST',  'Nationality',]
preparator = IsBadBuy_prepare_cols()
features_train = preparator.transform(features_train)
features_test = preparator.transform(features_test)


In [ ]:
## Split infromation in cols Model in SubModel (normalization)
cleaner = Cleaner()
features_train = cleaner.transform(features_train)
features_test = cleaner.transform(features_test)

In [ ]:
features_train.columns

In [ ]:
## Imputer remooved from the concideration, it makes predictions worse.

# from avp_pckg.IsBadBuy_imputePrices import IsBadBuyImputer
# imputer = IsBadBuyImputer(percent=1, verbose=0)
# df_tmp = pd.concat([features_train, features_test])
# imputer.fit(df_tmp)
# features_train = imputer.transform(features_train)
# features_test = imputer.transform(features_test)

## 2. Tree-model
cross-validate 

In [ ]:
# execution time= 34 s 
# from avp_pckg.avp_model_selection import cross_validate_pipe 
param_name ='max_depth'
param_range = [3, 4, 5, 6, 7, # 8, 9, 10, #12, 14, 16, 18,
               ]
score_dict = cross_validate_transformer(X=features_train,
                                y=target_train,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='tree',
                                n_jobs=-1,
                                )

plot_scores(score_dict, param_name='max_depth tree')
print_scores(score_dict)


## 2. Random Forest-model with whole dataset
cross-validate 

In [ ]:
# execution time= 7m 30s 
param_name ='max_depth'
param_range = [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 
               #16, 18, 20, 25, 30
               ]
score_dict = cross_validate_transformer(X=features_train,
                                y=target_train,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='forest', # 'forest', 'logistic', 'tree',
                                n_jobs=-1,
                                # cols_binary = cols_binary,
                                )

plot_scores(score_dict, param_name=param_name)
print_scores(score_dict)

## 3. Logistic Regresion - model with whole dataset
cross-validate 

In [ ]:
# execution time= 15s 
param_name ='C'
param_range = [0.001, 0.002, 0.004, 0.01, 0.02, 0.04, 0.1, 1, 10,]
score_dict = cross_validate_transformer(X=features_train,
                                y=target_train,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='logistic', # 'forest', 'logistic', 'tree',
                                n_jobs=-1,
                                # cols_binary = cols_binary,
                                )

plot_scores(score_dict, param_name=param_name, xlog=True)
print_scores(score_dict)

## 4. Support Vector Machine Classifier
cross-validate 

In [ ]:
# execution time= 43m
param_name ='C'
param_range = [
   # 0.001, 0.002, 0.004, 
    0.01, 0.1, 1, 
    # 10,
    ]
score_dict = cross_validate_transformer(X=features_train,
                                y=target_train,
                                param_name=param_name,
                                param_range= param_range,
                                cv=5, 
                                max_cat=25,
                                estimator_name='svc', # 'forest', 'logistic', 'tree',
                                n_jobs=-1,
                                # cols_binary = cols_binary,
                                )

plot_scores(score_dict, param_name=param_name, xlog=True)
print_scores(score_dict)

In [ ]:
## too small f1 score
# ## run time: 22 min
# param_name = 'n_neighbors'
# param_range = [ 3, 4, 5, 6, 7, 8, 9, 10
#     ]
# score_dict = cross_validate_transformer(X=features_train,
#                                 y=target_train,
#                                 param_name=param_name,
#                                 param_range= param_range,
#                                 cv=5, 
#                                 max_cat=25,
#                                 estimator_name='kn', # 'forest', 'logistic', 'tree',
#                                 n_jobs=-1,
#                                 kwards_fixed = {'weights':'uniform', 'n_jobs': -1}
#                                 )

# plot_scores(score_dict, param_name=param_name, xlog=True)
# print_scores(score_dict)


# Predictoins

In [ ]:

### Tree model #########################################################
pipe_tree = Pipeline(steps=[
('base', StandardOHETransformer(max_cat=25) ),
('model', DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=4))
])

pipe_tree.fit(features_train, target_train['IsBadBuy'].to_numpy())

pred_tree = pipe_tree.predict(features_test)
df_pred_tree = pd.DataFrame(pred_tree, index=features_test.index)
df_pred_tree.columns = ['tree']

report = classification_report(target_test, pred_tree)
print('tree report: \n', report)
print('tree pred.sum():', pred_tree.sum())



In [ ]:
importances = pipe_tree.named_steps.model.feature_importances_ 
feature_names = pipe_tree.named_steps.base.get_feature_names_out()
tree_importances = pd.Series(importances, index=feature_names)
tree_importances.sort_values(ascending=False).head(20)

In [ ]:

### Forest model #########################################################
pipe_forest = Pipeline(steps=[
('base', StandardOHETransformer(max_cat=25)),
('model', RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=7))
])

pipe_forest.fit(features_train, target_train['IsBadBuy'].to_numpy())
pred_forest = pipe_forest.predict(features_test)
df_pred_forest = pd.DataFrame(pred_forest, index=features_test.index)
df_pred_forest.columns = ['forest']


report = classification_report(target_test, pred_forest)
print('forest report: \n', report)
print('forest pred.sum():', pred_forest.sum())


In [ ]:
importances = pipe_forest.named_steps.model.feature_importances_ 
feature_names = pipe_tree.named_steps.base.get_feature_names_out()
tree_importances = pd.Series(importances, index=feature_names)
tree_importances.sort_values(ascending=False).head(50)

In [ ]:
### Logistic #########################################################
pipe_reg = Pipeline(steps=[
('preprocessing', StandardOHETransformer(max_cat=25)),
('model', LogisticRegression(class_weight='balanced', random_state=42, C=0.02))
])

pipe_reg.fit(features_train, target_train['IsBadBuy'].to_numpy())
pred_reg = pipe_reg.predict(features_test)
df_pred_reg = pd.DataFrame(pred_reg, index=features_test.index)
df_pred_reg.columns = ['logistic']

report = classification_report(target_test, pred_reg)
print('regression report: \n', report)
print('logistig pred.sum():', pred_reg.sum())



In [ ]:
## run time 12min
### SVC #########################################################
pipe_svc = Pipeline(steps=[
('preprocessing', StandardOHETransformer(max_cat=25)),
('model', SVC(class_weight='balanced', random_state=42, C=0.1))
])

pipe_svc.fit(features_train, target_train['IsBadBuy'].to_numpy())
pred_svc = pipe_svc.predict(features_test)
df_pred_svc = pd.DataFrame(pred_svc, index=features_test.index)
df_pred_svc.columns = ['svc']

report = classification_report(target_test, pred_svc)
print('regression report: \n', report)
print('pred.sum():', pred_svc.sum())

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# ### KNeighborsClassifier #########################################################
# pipe_kn = Pipeline(steps=[
# ('preprocessing', StandardOHETransformer(max_cat=25)),
# ('model', KNeighborsClassifier(n_neighbors=3, p=2, weights='uniform', leaf_size=100, n_jobs=-1))
# ])

# pipe_kn.fit(features_train, target_train['IsBadBuy'].to_numpy())
# pred_kn = pipe_kn.predict(features_test)
# df_pred_kn = pd.DataFrame(pred_kn, index=features_test.index)
# df_pred_kn.columns = ['kn']

# report = classification_report(target_test, pred_kn)
# print('regression report: \n', report)
# print('logistig pred.sum():', pred_svc.sum())

## Ensamble

In [ ]:
## check function for unanimity vote
from avp_pckg.small_functions import unanimity
pred_lst = [df_pred_tree, df_pred_forest, df_pred_reg,  df_pred_svc, ] # df_pred_kn
result = unanimity(pred_lst)

report = classification_report(target_test, result)
print('result report: \n', report)

## Prepare workflow for prediction of aim data

original train data contains target column 'IsBadBuy', therefore
- First step is constract similar structure from features_train (origenal train data contained target column)
- Second step is to build prediction workflow 
- Third step is to check with feature_test the total workflow
- Forth step is to make final prediction on aim data (test.csv)

In [ ]:
## step 1. constract similar structure from features_train
features_train = pd.read_csv(path_absolute + '\\data\\features_train.csv', parse_dates=['PurchDate'], index_col=0)
features_test = pd.read_csv(path_absolute + '\\data\\features_test.csv', parse_dates=['PurchDate'], index_col=0)

target_train = pd.read_csv(path_absolute + '\\data\\target_train.csv', index_col=0)
target_test = pd.read_csv(path_absolute + '\\data\\target_test.csv', index_col=0)

train =  features_train.join(target_train, 
                             sort='index'
                             )
test = features_test
train.head()

In [ ]:
## step 2. constract prediction workflow 

col_target = 'IsBadBuy'

def basis_model_predict(train:pd.DataFrame, test, col_target:str ):
    
    ## preprocessing 
    target = train['IsBadBuy'].to_numpy()
    features_train = train.drop(columns=[col_target])
    features_test = test
    
    preparator = IsBadBuy_prepare_cols()
    features_train = preparator.transform(features_train)
    features_test = preparator.transform(features_test)
    cleaner = Cleaner()
    features_train = cleaner.transform(features_train)
    features_test = cleaner.transform(features_test)
    
    prepare_cols = StandardOHETransformer(max_cat=25)  
    mtrx_train = prepare_cols.fit_transform(features_train)
    mtrx_test = prepare_cols.transform(features_test)

    ## models
    model_tree =  DecisionTreeClassifier(class_weight='balanced', random_state=42, max_depth=4)
    model_forest = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=7)
    model_logreg = LogisticRegression(class_weight='balanced', random_state=42, C=0.02)
   # model_svc = SVC(class_weight='balanced', random_state=42, C=0.1)

    model_tree.fit(mtrx_train, target)
    print('model tree fit finished')
    model_forest.fit(mtrx_train, target)
    print('model forest fit finished')
    model_logreg.fit(mtrx_train, target)
    print('model logistic regression fit finished')
   # model_svc.fit(mtrx_train, target)
   # print('model SVC fit finished')

    pred_tree = pd.DataFrame(model_tree.predict(mtrx_test), index=features_test.index)
    pred_forest = pd.DataFrame(model_forest.predict(mtrx_test), index=features_test.index)
    pred_logreg = pd.DataFrame(model_logreg.predict(mtrx_test), index=features_test.index)
   # pred_svc = pd.DataFrame(model_svc.predict(mtrx_test), index=features_test.index)

    result = unanimity([pred_tree, pred_forest, pred_logreg,# pred_svc
                        ]) # 
    df_result = pd.DataFrame(result)
    # print(df_result.columns)
    df_result = df_result.rename(columns={'sum': col_target })
    return df_result



In [ ]:
result = basis_model_predict(train, test, col_target,)
# result.head()

## step 3. Check with feature_test the total workflow
report = classification_report(target_test, result) # .sort_index()
print('result report: \n', report)

In [ ]:
## step 4. The final prediction on aim data

data_train = pd.read_csv(path_absolute + '\\data\\DontGetKicked\\training.csv', 
                         parse_dates=['PurchDate'], 
                         index_col=0)
# data_train.loc[:, 'WheelTypeID'] = data_train['WheelTypeID'].astype(str)
# data_train.head()

data_test = pd.read_csv(path_absolute + '\\data\\DontGetKicked\\test.csv', 
                         parse_dates=['PurchDate'], 
                         index_col=0)
# data_test.loc[:, 'WheelTypeID'] = data_test['WheelTypeID'].astype(str)
data_test.head()

In [ ]:
pred_aim = basis_model_predict(data_train, data_test, col_target)
pred_aim.head()

In [ ]:
pred_aim.to_csv(path_absolute + '\\data\\Model_predictions\\model_clean.csv') # price-diff _SVC
